In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

import time

from pymongo import MongoClient

In [32]:
# tor

In [33]:
#from stem import 

In [34]:
#import socks
#import socket
#import requests

#socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr="127.0.0.1", port=9050)
#socket.socket = socks.socksocket

In [35]:
#print requests.get("http://icanhazip.com").text

In [2]:
#path_to_chromedriver = '/home/ubuntu/chromedriver'
#browser = webdriver.Chrome(executable_path = path_to_chromedriver)

#fp = webdriver.FirefoxProfile()
#fp.set_preference("http.response.timeout", 1)
#fp.set_preference("dom.max_script_run_time", 1)
#fp.set_preference("javascript.enabled", False)

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
firefox_capabilities = DesiredCapabilities.FIREFOX

firefox_capabilities['marionette'] = True
firefox_capabilities['binary'] = '/home/ubuntu/firefox/firefox'

capstone_profile = webdriver.FirefoxProfile('/home/ubuntu/.mozilla/firefox/a64vis0x.capstone')

browser = webdriver.Firefox(capstone_profile, capabilities=firefox_capabilities)

In [3]:
client = MongoClient('mongodb://localhost:27017/')

In [4]:
#logging in

In [5]:
url = 'https://www.goodreads.com/'
browser.get(url)

In [6]:
username = browser.find_element_by_id('userSignInFormEmail')
password = browser.find_element_by_id('user_password')

In [7]:
with open('login.txt') as f:
    login = f.read().split('\n')

username.send_keys(login[0])
password.send_keys(login[1])

In [8]:
signIn = browser.find_element_by_xpath("/html[@class='desktop no-touchevents cssanimations flexbox flexwrap csstransforms es5date es5array es5object strictmode es5function es5string json es5undefined es5syntax es5']/body[@id='signedOutHome']/div[@class='content']/div[@id='siteheader']/div[@class='mainContent']/div[@id='navBar']/div[@id='signInForm']/form[@id='sign_in']/div[@class='formBox'][3]/input[@class='gr-button gr-button--dark']")

In [9]:
signIn.click()

In [10]:
# now we are logged in

In [11]:
# getting a user's friends

In [12]:
curUserID = 7607437

In [13]:
url = 'https://www.goodreads.com/friend/user/' + str(curUserID)
browser.get(url)

In [14]:
friendCount = browser.find_element_by_class_name('smallText').text

In [15]:
friendCount

u'(showing 1-30 of 238)'

In [16]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [17]:
friendIDs = []

numFriends = int(friendCount[friendCount.rfind(' ') +  1 : friendCount.rfind(')')])
numPages = numFriends/30 + 1

for i in range(1,numPages+1):
    url = 'https://www.goodreads.com/friend/user/' + str(curUserID) + '?page=' + str(i)

    browser.get(url)
    test=Select(browser.find_element_by_tag_name('select'))
    test.select_by_index(0)
        
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    friendTable = soup.find(id='friendTable')
    
    friendHrefs = [link.get('href') for link in friendTable.findAll('a')]
    friendHrefs = filter(lambda s: s is not None, friendHrefs)
    
    friendComps = filter(lambda s: s[:14]=='/user/compare/', friendHrefs)
    friendIDs.extend([int(s[14:]) for s in friendComps])

    time.sleep(1)

In [18]:
if len(friendIDs) != numFriends:
    print 'user %d has %d friends, but retrieved %d friends instead' % (curUserID,numFriends,len(friendIDs))

In [19]:
# entering into mongodb

In [20]:
db = client['goodreads']

In [21]:
#db.create_collection('reviews')
#db.create_collection('friends')

In [22]:
friends = db['friends']

In [30]:
# add curUserID's friends
friends.update_one(
    {"userID": curUserID}, 
    {"$addToSet": {"friends": {"$each": friendIDs}}}, 
    upsert=True)

# add curUserID to f-list of each of their friends

for friend in friendIDs:
    friends.update_one(
    {"userID": friend}, 
    {"$addToSet": {"friends": curUserID}}, 
    upsert=True)


In [35]:
friends.find_one({"userID": friendIDs[1]})

{u'_id': ObjectId('58156aebb59515fedb9f7bb7'),
 u'friends': [7607437],
 u'userID': 1938805}